In [1]:
import requests

url = "https://web3.ncaa.org/aprsearch/gsrsearch"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36"
}

response = requests.get(url, headers=headers)

In [2]:
import os
import pandas as pd
import requests

In [3]:
import requests

url = "https://www.fws.gov/press-release/2023-10/21-species-delisted-endangered-species-act-due-extinction"

response = requests.get(url)
response
from bs4 import BeautifulSoup


soup_doc = BeautifulSoup(response.content, 'html.parser')
soup_doc.prettify()

'<!DOCTYPE html>\n<html dir="ltr" lang="en" prefix="og: https://ogp.me/ns#">\n <head>\n  <meta charset="utf-8"/>\n  <script async="" src="https://www.googletagmanager.com/gtag/js?id=G-KERW0RESPC">\n  </script>\n  <script>\n   window.dataLayer = window.dataLayer || [];function gtag(){dataLayer.push(arguments)};gtag("js", new Date());gtag("set", "developer_id.dMDhkMT", true);gtag("config", "G-KERW0RESPC", {"groups":"default","page_placeholder":"PLACEHOLDER_page_location"});\n  </script>\n  <meta content="FALLS CHURCH, Va. —\xa0 The U.S. Fish and Wildlife Service is delisting 21 species from the Endangered Species Act due to extinction. Based on rigorous reviews of the best available science for each of these species, the Service determined these species are extinct and should be removed from the list of species protected under the ESA." name="description"/>\n  <link href="https://www.fws.gov/press-release/2023-10/21-species-delisted-endangered-species-act-due-extinction" rel="canonical"/

In [4]:
table = soup_doc.find("table")
table

<table><tbody><tr><td><p><span><span><span><strong><span><span>Species Name</span></span></strong></span></span></span></p></td><td><p><span><span><span><strong><span><span>Found</span></span></strong></span></span></span></p></td><td><p><span><span><span><strong><span><span>Listed</span></span></strong></span></span></span></p></td><td><p><span><span><span><strong><span><span>Last Confirmed Sighting</span></span></strong></span></span></span></p></td><td><p><span><span><span><strong><span><span>Docket Number</span></span></strong></span></span></span></p></td></tr><tr><td><p><span><span><span><strong><span><span>MAMMALS</span></span></strong></span></span></span></p></td></tr><tr><td><p><span><span><span><span><span>Little Mariana fruit bat</span></span></span></span></span></p></td><td><p><span><span><span><span><span>GU (Guam)</span></span></span></span></span></p></td><td><p><span><span><span><span><span>1984</span></span></span></span></span></p></td><td><p><span><span><span><span

In [17]:
from playwright.async_api import async_playwright
import pandas as pd

async def scrape_fws_delisted_species():
    async with async_playwright() as p:
        browser = await p.chromium.launch()
    page = await (await browser.new_context()).new_page()
    await page.goto("https://www.fws.gov/press-release/2023-10/21-species-delisted-endangered-species-act-due-extinction")

In [25]:
from playwright.async_api import async_playwright
import pandas as pd

async def scrape_fws_delisted_species():
    async with async_playwright() as p:
        browser = await p.chromium.launch()
        page = await (await browser.new_context()).new_page()
        await page.goto("https://www.fws.gov/press-release/2023-10/21-species-delisted-endangered-species-act-due-extinction")
        await page.wait_for_selector("table")

        table = await page.query_selector("table")
        rows = await table.query_selector_all("tr")

        data = []
        max_cols = 0

        for species, row in enumerate(rows):
            cells = await row.query_selector_all("th" if species == 0 else "td")
            text = [await cell.inner_text() for cell in cells]
            if text:
                data.append(text)
                if len(text) > max_cols:
                    max_cols = len(text)

        await browser.close()

    for row in data:
        while len(row) < max_cols:
            row.append("")

    headers = data[0] if max_cols == len(data[0]) else [f"Column {i+1}" for i in range(max_cols)]
    df = pd.DataFrame(data[1:], columns=headers)
    df.to_csv("delisted_species.csv", index=False)
    print("delisted_species.csv")

await scrape_fws_delisted_species()

delisted_species.csv
